In [ ]:

!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp311-cp311-linux_x86_64.whl size=831161 sha256=06b4776fafecebacc0d14f1686b07b1d364b9b74774404a922e361cd4298b54d
  Stored in directory: /root/.cache/pip/wheels/b9/0d/8a/0729d2e6e3ca2a898ba55201f905da7db3f838a33df5b3fcdd
Successfully built lightfm


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import coo_matrix
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import coo_matrix

# dowload  dataset

In [ ]:
! pip install -q kaggle
from google.colab import files
files.upload()

Saving kaggle (3).json to kaggle (3) (1).json


{'kaggle (3) (1).json': b'{"username":"chokkapusaketh","key":"9e9e654009127293f44895caebb4959b"}'}

In [ ]:
!ls ~/.kaggle


ls: cannot access '/root/.kaggle': No such file or directory


In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mohamedbakhet/amazon-books-reviews")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/amazon-books-reviews


In [ ]:
!kaggle datasets download -d mohamedbakhet/amazon-books-reviews


Dataset URL: https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews
License(s): CC0-1.0


# unzip dataset

In [ ]:
!unzip amazon-books-reviews.zip


Archive:  amazon-books-reviews.zip
  inflating: Books_rating.csv        
  inflating: books_data.csv          


In [ ]:
import os
os.listdir()

['.config',
 'kaggle (3).json',
 'kaggle (3) (1).json',
 'Books_rating.csv',
 'amazon-books-reviews.zip',
 'books_data.csv',
 'sample_data']

In [ ]:
books_df=pd.read_csv('books_data.csv')
ratings_df=pd.read_csv('Books_rating.csv')

# 1. Check column names and nulls



In [ ]:
books_df.isnull().sum()


,0
Title,1
description,68442
authors,31413
image,52075
previewLink,23836
publisher,75886
publishedDate,25305
infoLink,23836
categories,41199
ratingsCount,162652


In [ ]:
ratings_df.isnull().sum()

,0
Id,0
Title,208
Price,2518829
User_id,561787
profileName,561905
review/helpfulness,0
review/score,0
review/time,0
review/summary,407
review/text,8


# ✅ Drop unnecessary and very sparse columns
We keep only what's useful for LightFM:

* User_id (user)

* Title or book_id (item)

* review/score (rating)

In [ ]:
# lodaing only nessary columsn from books data
books_df = books_df[["Title", "authors", "categories", "description", "ratingsCount"]]




```
# This is formatted as code
```

# 🔁 LabelEncoder — What we're doing
We are converting user IDs and book titles (which are strings) into numeric indices because ML models (like LightFM) work only with numbers.

In [ ]:
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()

ratings_df["user"] = user_encoder.fit_transform(ratings_df["User_id"])
ratings_df["item"] = item_encoder.fit_transform(ratings_df["Title"])
ratings_df["rating"] = ratings_df["review/score"]

In [ ]:
from scipy.sparse import coo_matrix
interactions = coo_matrix((
    ratings_df["rating"],
    (ratings_df["user"], ratings_df["item"])
))


# interactions: the matrix of user-item interactions.

* random_train_test_split: splits it into a training set and a test set.

* test_percentage=0.2: 20% of the data will be used for testing.

# spliting dataset to 80 and 20

In [ ]:
from lightfm.cross_validation import random_train_test_split

train, test = random_train_test_split(interactions, test_percentage=0.2)


In [ ]:
from lightfm import LightFM

model = LightFM(loss='warp')  # WARP is good for implicit data like ratings
model.fit(train, epochs=10, num_threads=4)



# For Precision@5, here's a rough idea of what's considered good — but it depends a lot on the dataset:

# Precision@5	Interpretation
* < 0.01	Very low — likely due to sparse data or model needing tuning.
* 0.01 – 0.05	Decent for sparse, large-scale datasets.
* 0.05 – 0.1	Good! Model is catching relevant items.
* > 0.1	Excellent — very rare unless the dataset is dense and well-tuned.
#📌 So, aim for at least 0.03+ for a practical recommender on sparse data like Amazon reviews.
* Your 0.008 is a starting point — we can improve it with better preprocessing, filtering, and model tuning.

# 1st evaluvation

In [ ]:
from lightfm.evaluation import precision_at_k

precision = precision_at_k(model, test, k=5).mean()
print(f'Precision@5: {precision:.4f}') #fun fact it took 3 hr to train


Precision@5: 0.0094


# Filter Active Users & Popular Books
* Remove users with very few reviews (e.g., <5).

* Remove books with very few ratings (e.g., <10).

This removes noise and sparsity.

# 🔧 Step 1: Filter Active Users & Popular Books
We’ll:

* Count how many reviews each user has written.

* Keep users with ≥ 5 reviews.

* Count how many times each book has been rated.

Keep books with ≥ 10 ratings.

In [ ]:
user_counts = ratings_df['User_id'].value_counts() # Count user reviews
active_users = user_counts[user_counts >= 5].index

book_counts = ratings_df['Title'].value_counts() # Count book ratings
popular_books = book_counts[book_counts >= 10].index

filtered_df = ratings_df[
    (ratings_df['User_id'].isin(active_users)) & # Filter the ratings dataframe
    (ratings_df['Title'].isin(popular_books))
]

print(f"Original ratings: {ratings_df.shape[0]}")
print(f"Filtered ratings: {filtered_df.shape[0]}")


Original ratings: 3000000
Filtered ratings: 1018791


# Rebuild the Interaction Matrix
We'll now:

* Use the filtered_df to recreate the interaction matrix.

* This time it’ll be smaller = faster training, less RAM, and potentially better quality.

In [ ]:
from lightfm.data import Dataset

dataset = Dataset()
dataset.fit(filtered_df['User_id'], filtered_df['Title']) # Fit the users and items
(interactions, weights) = dataset.build_interactions( # Build interactions matrix
    [(x['User_id'], x['Title'], float(x['review/score'])) for _, x in filtered_df.iterrows()]
)


# 2n evaluvation

In [ ]:
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k

# Step 1: Split into training and testing sets
train, test = random_train_test_split(interactions, test_percentage=0.2)

# Step 2: Initialize model
model = LightFM(loss='warp')  # WARP focuses on ranking

# Step 3: Fit the model
model.fit(train, epochs=10, num_threads=4)

# Step 4: Evaluate
precision = precision_at_k(model, test, k=5).mean()
print(f"Precision@5: {precision:.4f}")


Precision@5: 0.0818


In [ ]:
model = LightFM(loss='warp', learning_rate=0.05, no_components=50)
model.fit(train, epochs=40, num_threads=6)


# Great question. Here's what I suggest, in order of impact vs. effort:

* ✅ 1. Try WARP loss instead of BPR
WARP (Weighted Approximate-Rank Pairwise) is often better for ranking precision.

* Just change:

* model = LightFM(loss='warp')
*➡️ Effort: Minimal
*📈 Impact: Often improves Precision@K



# 🏆 Best Performing Model: LightFM with WARP Loss
Reason: Among all models evaluated, the WARP (Weighted Approximate-Rank Pairwise) model showed the highest performance.

* Precision@K: 35.6%

* This means approximately 1 in 3 recommendations were relevant to the user — making it the best choice for our current system.

In [ ]:
from lightfm.evaluation import precision_at_k

# Evaluate on the test set using Precision@5
precision = precision_at_k(model, test, k=5).mean()
print(f"Precision@5: {precision:.4f}")


Precision@5: 0.1683


# Distance in Precision@5
* You're at 0.1688
# → You’re ~0.03 away from hitting 0.20, which is considered very good for most practical LightFM systems.

In [ ]:
import gzip
import pickle

with gzip.open("best_model.pkl.gz", "wb") as f:
    pickle.dump(model, f, protocol=pickle.HIGHEST_PROTOCOL)

print("✅ Compressed model saved!")


✅ Compressed model saved!


In [ ]:
with gzip.open("best_model.pkl.gz", "rb") as f:
    model = pickle.load(f)


In [ ]:
with gzip.open("best_model.pkl.gz", "rb") as f:
    model = pickle.load(f)


In [ ]:
import gzip
import pickle

with gzip.open("best_model.pkl.gz", "rb") as f:
    model = pickle.load(f)

# Check model parameters
print(model)


In [ ]:
print("Loss function:", model.loss)
print("No. of latent factors:", model.no_components)



Loss function: warp
No. of latent factors: 50


In [ ]:
pip install huggingface_hub


In [ ]:
from huggingface_hub import login

login()  # Paste your HF token when prompted (get it from https://huggingface.co/settings/tokens)


In [ ]:
from huggingface_hub import HfApi

api = HfApi()

api.upload_file(
    path_or_fileobj="best_model.pkl.gz",
    path_in_repo="best_model.pkl.gz",
    repo_id="cs22/cs22_bk_recommendation",  # 👈 use your actual username + repo name
    repo_type="model"
)


best_model.pkl.gz:   0%|          | 0.00/46.1M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/cs22/cs22_bk_recommendation/commit/87c6cbffb5109892e4f39431e38cc99af2c90783', commit_message='Upload best_model.pkl.gz with huggingface_hub', commit_description='', oid='87c6cbffb5109892e4f39431e38cc99af2c90783', pr_url=None, repo_url=RepoUrl('https://huggingface.co/cs22/cs22_bk_recommendation', endpoint='https://huggingface.co', repo_type='model', repo_id='cs22/cs22_bk_recommendation'), pr_revision=None, pr_num=None)